In [ ]:
# stream_graph_updates関数をリファクタリングし、stream_graph_updates(user_input)で呼び出せる形に修正
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from typing import Annotated
from typing_extensions import TypedDict
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
# AIMessageをインポートして、メッセージの種類を判定できるようにします
from langchain_core.messages import AIMessage

# ===== Stateクラスの定義 =====
class State(TypedDict):
    messages: Annotated[list, add_messages]

# ===== グラフの構築 =====
def build_graph(model_name):
    # ソースコードを記述    
    # 1. ツールを定義
    tool = TavilySearchResults(max_results=2)
    tools = [tool]

    # 2. グラフのインスタンスを作成
    graph_builder = StateGraph(State)

    # 3. 言語モデルを定義し、ツールを紐づけ
    llm = ChatOpenAI(model_name=model_name)
    llm_with_tools = llm.bind_tools(tools)

    # 4. chatbotノードを作成
    def chatbot(state: State):
        return {"messages": [llm_with_tools.invoke(state["messages"])]}
    
    # 5. グラフにノードを追加
    graph_builder.add_node("chatbot", chatbot)
    tool_node = ToolNode(tools)
    graph_builder.add_node("tools", tool_node)

    # 6. グラフにエッジを追加
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
    )
    graph_builder.add_edge("tools", "chatbot")
    
    # 7. 開始点を設定
    graph_builder.set_entry_point("chatbot")

    # 8. 記憶（メモリ）を持たせてグラフをコンパイル
    memory = MemorySaver()
    graph = graph_builder.compile(checkpointer=memory)
    
    return graph

# ===== グラフ実行関数 =====
def stream_graph_updates(user_input: str):
    # ソースコードを記述
    # スレッディングを使用して、会話履歴を保持
    config = {"configurable": {"thread_id": "1"}}
    
    # すでに出力された内容を保持する変数
    printed_content = ""
    
    # ストリーミングでグラフを実行し、結果を表示
    events = graph.stream(
        {"messages": [("user", user_input)]}, config, stream_mode="values"
    )
    for event in events:
        # 最新のメッセージを取得
        latest_message = event["messages"][-1]
        
        # 最新のメッセージがAIMessageの場合のみ、その内容を表示します
        if isinstance(latest_message, AIMessage):
            # 新しく追加された内容のみを抽出
            new_content = latest_message.content[len(printed_content):]
            # 新しい内容を追記して出力
            print(new_content, end="", flush=True)
            # 出力済み内容を更新
            printed_content = latest_message.content


# ===== メイン実行ロジック =====
# 環境変数の読み込み
load_dotenv("../.env")
os.environ['OPENAI_API_KEY'] = os.environ['API_KEY']
os.environ['TAVILY_API_KEY'] = os.environ.get('TAVILY_API_KEY')

# モデル名
MODEL_NAME = "gpt-4o-mini" 

# グラフの作成
graph = build_graph(MODEL_NAME)

# メインループ
while True:
    user_input = input("あなた: ")
    print(f"あなた: {user_input}") 
    if user_input.strip() == "":
        print("AI: ご利用ありがとうございました！")
        break
    # AIの応答であることを示すためにプロンプトを表示
    print("AI: ", end="")
    stream_graph_updates(user_input)
    # 回答がすべて表示された後に、バッファに溜めずに即時に改行
    print("\n", flush=True)


あなた: こんにちは
AI: こんにちは！今日はどんなことをお手伝いできますか？

あなた: １たす２は？
AI: 1たす2は3です。何か他にお手伝いできることがありますか？

あなた: 台湾観光について検索結果を教えて
AI: 台湾観光に関する情報をいくつか見つけました。

1. **Hankyu Travel**の情報:
   - 台湾は絶景、グルメ、ショッピング、温泉など多彩な楽しみ方ができる観光地です。台北101でのショッピングや眺望、寧夏夜市でのB級グルメ、九份の街歩きなどが特におすすめです。
   - おすすめ観光スポット:
     - **九份**: 映画『千と千尋の神隠し』の舞台として知られる町。
     - **士林夜市**: 台北市最大の観光夜市で、ローカルフードが楽しめます。
     - **日月潭**: 台湾最大の淡水湖で、美しい景観が魅力。
     - **台北101**: 高さ508mの超高層ビル。
     - **龍虎塔**: 高雄にある伝説的な塔。
     - **太魯閣溪谷**: 自然豊かな渓谷。

   詳細は[こちら](https://www.hankyu-travel.com/guide/taiwan/)から。

2. **近畿日本ツーリスト**の情報:
   - 台湾は日本から直行便で約4時間、週末や連休に訪れやすい観光地です。台北の定番スポットとして、台北101、国立故宮博物院、士林観光夜市などが挙げられます。
   - その他のおすすめスポット:
     - **淡水**: 港町で美しい景色が広がっています。
     - **迪化街**: レトロな街並みが残る場所。
     - **永康街**: 名店が並ぶ美食エリア。

   詳細は[こちら](https://www.knt.co.jp/travelguide/kaigai/027/)から。

台湾には多くの魅力的な観光地がありますので、ぜひ訪れてみてください！他に知りたいことがあれば教えてください。

あなた: 
AI: ご利用ありがとうございました！
